# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902069


The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.

In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:07<01:43,  3.58s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<01:08,  2.55s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:47,  1.84s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:33,  1.32s/it]

...

Rendering models:  84%|████████▍ | 26/31 [00:10<00:00,  5.38it/s]

Rendering models:  87%|████████▋ | 27/31 [00:11<00:00,  4.93it/s]

Rendering models:  94%|█████████▎| 29/31 [00:11<00:00,  6.26it/s]

Rendering models:  97%|█████████▋| 30/31 [00:11<00:00,  6.42it/s]

equidad1                              0.083934
Linda_J._Berkel                       0.000138
not-logged-in-0b0736c8991a7cc2017a    0.000172
not-logged-in-03b60317d6ee47942c58    0.001015
not-logged-in-dacb6c2f9eb6080c5e9f    0.010751
not-logged-in-8c0c44faf754d344b18a    0.000171
Thedunkmasta                          0.000774
ElisabethB                            0.071739
47degreesnorth                        0.000202
Alexander00                           0.000262
pollux2728                            0.009954
wrxtminecrafter                       0.001535
not-logged-in-0a609d3e5fdaf174d69a    0.007415
rudolf_wieczorek                      0.000521
AdamMorse                             0.000194
magedghoche                           0.000384
not-logged-in-1fd5911ffad23c244f1a    0.002306
not-logged-in-314d48142a3a122ba666    0.000759
AlexYoung35                           0.000319
pangeli5                              0.000620
not-logged-in-214814e52004b42491a4    4.001722
acapirala    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())